<a href="https://colab.research.google.com/github/SamuelBFG/DL-studies/blob/master/3_tf_explain_catdog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# `tf-explain`

tf-explain (by Raphael Meudec) helps understand the results and inner workings of a neural network with the help of visualizations, removing the veil on bias in our datasets. Few different visualization approaches are available with tf.explain.

- Grad CAM: The Gradient-weighted Class Activation Mapping visualizes how parts of the image affect the neural network's output by looking into the activation maps. A heatmap is generated based on the gradients of the object id from the last convolutional layer. Grad CAM is largely a broad-spectrum heatmap generator as it is robust to noise and can be used on an array of CNN models. 
- Activations: Visualize the activations for the convolutional layers. 
- Occlusion Sensitivity: Occludes a part of the image (using a small square patch placed randomly) to figure out how robust the network is. If the prediction is still correct, on average, the network is robust. The area in the image that is the warmest (i.e. red) has the most effect on the prediction when occluded.

In this notebook we will produce different visualizations on the sample images.

Note: After executing the first cell you may have to `RESTART RUNTIME` if you are running on Google Colab.

In [10]:
# Perform all installations
!pip install tensorflow-gpu==2.0.0
!pip install tf-explain==0.1.0

In [11]:
import tensorflow as tf
import numpy as np

#Get TensorBoard to run
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


We can add multiple types of callbacks while training or use its core API to generate TensorFlow events that can later be loaded into TensorBoard. 

In [12]:
from tf_explain.core.grad_cam import GradCAM
from tf_explain.core.occlusion_sensitivity import OcclusionSensitivity
from tf_explain.core.activations import ExtractActivations

In [13]:
#For inference, all we need to do is pass an image, its ImageNet object ID along with a model into tf-explain’s functions. The object id is needed as tf.explain needs to figure out what is activated for that particular class.

In [14]:
try:
  import google.colab
  IS_COLAB_ENV = True
except:
  IS_COLAB_ENV = False

In [15]:
def download_sample_image(filename):
    import requests
    url = f'https://raw.githubusercontent.com/PracticalDL/Practical-Deep-Learning-Book/master/sample-images/{filename}'
    open(filename, 'wb').write(requests.get(url).content)

In [16]:
if IS_COLAB_ENV:
    IMAGE_PATHS = ['dog.jpg', 'cat.jpg']
    for each_filename in IMAGE_PATHS:
        download_sample_image(each_filename)
else:
    IMAGE_PATHS = [ '/content/dog.jpg', '/content/cat.jpg']

In [17]:
indices = [263, 281]

layers_name = ['activation_6']

for i in range(len(IMAGE_PATHS)):
    each_path = IMAGE_PATHS[i]
    index = indices[i]

    img = tf.keras.preprocessing.image.load_img(each_path,
                                                target_size=(224, 224))
    img = tf.keras.preprocessing.image.img_to_array(img)
    data = ([img], None)
    # Define name with which to save the result as
    name = each_path.split("/")[-1].split(".jpg")[0]

    #Save the Grad Cam visualization
    explainer = GradCAM()
    model = tf.keras.applications.vgg16.VGG16(weights='imagenet',
                                              include_top=True)
    grid = explainer.explain(data, model, 'block5_conv3', index)
    explainer.save(grid, 'files', name + 'grad_cam.png')

    #grid = explainer.explain(data, model, 'block5_conv3', index)
    #explainer.save(grid, catpath, name + 'grad_cam.png')

    # Save the Occlusion Sensitivity visualization
    explainer = OcclusionSensitivity()
    model = tf.keras.applications.resnet50.ResNet50(weights='imagenet',
                                                    include_top=True)
    # Compute Occlusion Sensitivity for patch_size 20
    grid = explainer.explain(data, model, index, 20)
    explainer.save(grid, 'files', name + 'occlusion_sensitivity_20.png')
    # Compute Occlusion Sensitivity for patch_size 10
    grid = explainer.explain(data, model, index, 10)
    explainer.save(grid, 'files', name + 'occlusion_sensitivity_10.png')

    # Save the Activations visualizations
    data = (np.array([img]), None)
    explainer = ExtractActivations()
    model = tf.keras.applications.mobilenet.MobileNet(weights='imagenet',
                                                      include_top=True)
    grid = explainer.explain(data, model, ['conv1'])
    explainer.save(grid, 'files', name + 'activations.png')

Please donwload the folder ***files*** that was created.

In [21]:
!cd /content
# Zip the directory so that we can download it
!zip -r /content/files.zip /content/files

updating: content/files/ (stored 0%)
updating: content/files/catocclusion_sensitivity_10.png (deflated 0%)
updating: content/files/catactivations.png (deflated 0%)
updating: content/files/dogocclusion_sensitivity_10.png (deflated 0%)
updating: content/files/doggrad_cam.png (deflated 0%)
updating: content/files/catgrad_cam.png (deflated 0%)
updating: content/files/catocclusion_sensitivity_20.png (deflated 0%)
updating: content/files/dogocclusion_sensitivity_20.png (deflated 0%)
updating: content/files/dogactivations.png (deflated 0%)
